In [1]:
import hotspot
from itertools import groupby
from json import load
import pandas as pd
import scanpy as sc
import numpy as np
import sys
import modules.classifyClusters.classifyClusters as classify
import os
from matplotlib import pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns

adata = sc.read("./output/savedDataClustersFinal.h5ad")
adata.var_names_make_unique()
adata.X = adata.X.astype('float64')

sc.pp.filter_genes(adata, min_counts=3)

sc.pp.normalize_total(adata, target_sum=10000)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)

adata.raw = adata
adata = adata[:, adata.var.highly_variable]

sc.pp.regress_out(adata, ['total_counts', 'pct_counts_mt'])
sc.pp.scale(adata, max_value=10)
sc.tl.pca(adata, svd_solver='arpack')
sc.pp.neighbors(adata, n_neighbors=15, n_pcs=40) # n_neighbors=15 is default
sc.tl.umap(adata)
sc.tl.leiden(adata, resolution=0.3)

/var/folders/ts/m7c_798d70q3whk59gstkgz00000gn/T/ipykernel_9047/137862686.py:27: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  sc.pp.regress_out(adata, ['total_counts', 'pct_counts_mt'])


In [2]:
adataHotspot = sc.read("./output/savedDataClustersFinal.h5ad")
adataHotspot.var_names_make_unique()
adataHotspot.X = adataHotspot.X.astype('float64')
adataHotspot.obsm["X_pca"] = adata.obsm["X_pca"]
print(adataHotspot)
sc.pp.filter_genes(adataHotspot, min_counts=50)
print(adataHotspot)

AnnData object with n_obs × n_vars = 34626 × 36601
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'group', 'sample', 'batch', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'pct_counts_ribo', 'n_genes', 'leiden'
    var: 'features', 'mt', 'ribo', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'
    obsm: 'X_pca'
AnnData object with n_obs × n_vars = 34626 × 23606
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'group', 'sample', 'batch', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'pct_counts_ribo', 'n_genes', 'leiden'
    var: 'features', 'mt', 'ribo', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'n_counts'
    obsm: 'X_pca'


In [3]:
hs = hotspot.Hotspot(
    adataHotspot,
    model='danb',
    latent_obsm_key="X_pca",
)

/Users/greencode/Library/Python/3.9/lib/python/site-packages/hotspot/hotspot.py:98: UserWarning: Hotspot will work faster when counts are a csc sparse matrix.
  warnings.warn(


In [4]:
hs.create_knn_graph(weighted_graph=False, n_neighbors=30)

In [5]:
hs_results = hs.compute_autocorrelations()

 30%|███       | 7150/23606 [1:48:18<4:09:17,  1.10it/s]   


KeyboardInterrupt: 

In [ ]:
hs.results

In [ ]:
hs_genes = hs_results.loc[hs_results.FDR < 0.05].index # Select genes
local_correlations = hs.compute_local_correlations(hs_genes, jobs=1) # jobs

In [ ]:
hs_genes

In [ ]:
local_correlations

In [ ]:
modules = hs.create_modules(
    min_gene_threshold=30, core_only=True, fdr_threshold=0.05
)

In [ ]:
hs.plot_local_correlations()